In [1]:
import os
os.chdir('../')

In [2]:
%pwd

'c:\\Users\\bilal\\Documents\\mlops\\Wine-Quality'

In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainingConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str




In [5]:
from src.WineQuality.constants import *
from src.WineQuality.utils.common import read_yaml, create_directories


In [10]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self) -> ModelTrainingConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainingConfig(
            root_dir = Path(config.root_dir),
            train_data_path = Path(config.train_data_path),
            test_data_path = Path(config.test_data_path),
            model_name = config.model_name,
            alpha = params.alpha,
            l1_ratio = params.l1_ratio,
            target_column = schema.name
        )

        return model_trainer_config

In [11]:
import pandas as pd
import os
from src.WineQuality import logging
from sklearn.linear_model import ElasticNet
import joblib

In [12]:
class ModelTrainer:
    def __init__(self, config: ModelTrainingConfig):
        self.config = config

    def train(self):
        logging.info("Loading training data")
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)
        train_x=train_data.drop(columns=[self.config.target_column], axis=1)
        test_x=test_data.drop(columns=[self.config.target_column], axis=1)
        train_y=train_data[self.config.target_column]
        test_y=test_data[self.config.target_column]

        lr=ElasticNet(
            alpha=self.config.alpha,
            l1_ratio=self.config.l1_ratio,
            random_state=42
        )
        lr.fit(train_x, train_y)
        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))

In [13]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e


[2025-05-13 02:23:32,954: INFO:wine-quality:27: YAML file config\config.yaml loaded successfully.]
[2025-05-13 02:23:32,956: INFO:wine-quality:27: YAML file params.yaml loaded successfully.]
[2025-05-13 02:23:32,959: INFO:wine-quality:27: YAML file schema.yaml loaded successfully.]
[2025-05-13 02:23:32,960: INFO:wine-quality:48: Creating directory: artifacts]
[2025-05-13 02:23:32,961: INFO:wine-quality:48: Creating directory: artifacts/model_trainer]
[2025-05-13 02:23:32,962: INFO:root:6: Loading training data]


FileNotFoundError: [Errno 2] No such file or directory: 'artifacts\\model_trainer\\train.'